# r/wallstreetbets Text Generation using GPT-2
## Using `aitextgen`

In [2]:
# Setup
from aitextgen.TokenDataset import TokenDataset
from aitextgen.tokenizers import train_tokenizer
from aitextgen import aitextgen
import pandas as pd
import math
import re

Pull in the data.

In [3]:
wsb = pd.read_csv("./wsbsentiment.csv", names = ['title', 'text', 'sentiment'], encoding = "utf-8", encoding_errors = 'ignore')
wsbstrlist = []
for index, row in wsb.iterrows():
    wsbstrlist.append(str(row['title']))
    wsbstrlist.append(str(row['text']))
wsbstrlist = [element for element in wsbstrlist if element != 'nan']
for element in range(len(wsbstrlist)):
    result = re.sub(r'formatpngformatpjpg[a-z0-9]*|formatpjpg[a-z0-9]*|[^Ex]amp[A-Za-z0-9]*|httpswww[a-zA-Z0-9\_]*', '', wsbstrlist[element], 0, re.MULTILINE)
    if result:
        wsbstrlist[element] = result
with open('wsb_text.txt', 'w', encoding = 'utf-8', errors = 'replace') as f:
    for i in range(0, math.floor(len(wsbstrlist))):
        f.write(wsbstrlist[i].strip() + '\n')
f.close()
file_name = "wsb_text.txt"

Train a custom BPE tokenizer on the text. This will save one file `aitextgen.tokenizer.json`, which contains the information needed to rebuild the tokenizer.

In [4]:
train_tokenizer(file_name)
tokenizer_file = "aitextgen.tokenizer.json"

Check for CUDA.

In [5]:
!nvidia-smi

Tue May  3 20:24:59 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 511.79       Driver Version: 511.79       CUDA Version: 11.6     |
|-------------------------------+----------------------+----------------------+
| GPU  Name            TCC/WDDM | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Quadro M6000       WDDM  | 00000000:08:00.0  On |                    0 |
| 26%   38C    P8    21W / 250W |    563MiB / 11520MiB |     19%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

Instantiate `aitextgen` using the created tokenizer.

In [6]:
ai = aitextgen(tf_gpt2 = "355M", to_gpu = True)

Build a dataset for training by creating `TokenDataset`s, which automatically processes the dataset with the appropriate size.

In [7]:
data = TokenDataset(file_name, tokenizer_file = tokenizer_file, block_size = 64)

  0%|          | 0/1107 [00:00<?, ?it/s]

Train the model. This will save `pytorch_model.bin` periodically and after completion to the `trained_model` folder.

In [8]:
ai.train(data, batch_size = 8, num_steps = 25000, generate_every = 5000, save_every = 5000, max_split_size_mb = 8)

pytorch_model.bin already exists in /trained_model and will be overwritten!
Windows does not support multi-GPU training. Setting to 1 GPU.
C:\Users\kim3\AppData\Roaming\Python\Python310\site-packages\pytorch_lightning\trainer\connectors\callback_connector.py:151: LightningDeprecationWarning: Setting `Trainer(checkpoint_callback=False)` is deprecated in v1.5 and will be removed in v1.7. Please consider using `Trainer(enable_checkpointing=False)`.
  rank_zero_deprecation(
C:\Users\kim3\AppData\Roaming\Python\Python310\site-packages\pytorch_lightning\trainer\connectors\callback_connector.py:96: LightningDeprecationWarning: Setting `Trainer(progress_bar_refresh_rate=20)` is deprecated in v1.5 and will be removed in v1.7. Please pass `pytorch_lightning.callbacks.progress.TQDMProgressBar` with `refresh_rate` directly to the Trainer's `callbacks` argument instead. Or, to disable the progress bar pass `enable_progress_bar = False` to the Trainer.
  rank_zero_deprecation(
C:\Users\kim3\AppData\

  0%|          | 0/25000 [00:00<?, ?it/s]

C:\Users\kim3\AppData\Roaming\Python\Python310\site-packages\pytorch_lightning\trainer\trainer.py:2264: LightningDeprecationWarning: `trainer.progress_bar_dict` is deprecated in v1.5 and will be removed in v1.7. Use `ProgressBarBase.get_metrics` instead.
  rank_zero_deprecation(


Reload the trained model.

In [20]:
ai = aitextgen(model_folder="trained_model",
               tokenizer_file="aitextgen.tokenizer.json",
               to_gpu=True)

Generate some text!

In [21]:
ai.generate_one(temperature = 0.5, top_p = 0.9)

'or dangerous.\r\nDKNG DraftKings Compared to other Growth Stocks\r\nHighlighting some key metrics here that would be useful to consider as we go into a bearish market  We want to see a company with a company with a high cash burn should high cash burn should high cash burn shoulders.   Weight of cap range.   We are seeing adders.   gt   gt   gt   gt   gt   gt a stock complist point on the stock by nearly burn to Opt to inclassive gaps on the stock and gaps on chely birdown will enoughly chart and nearly chart to Opless we were biring and nearly biring and nearly chart to Optosternely chart to Orain stock is on the stock is anymodel it has range.   gt   gt   gt   gt   gt   gt   gt   gt   gt   gt   gt   gt   gt   gt   gt   gt   gt   gt'